In [3]:
## libreries needed
import pandas as pd
import geopandas as gpd
import pandas as pd
import shapely as shape
import os
import fiona
import sqlite3

In [2]:
# Functions used in the analysis
def CreateGDF_Points (puntos, names=["Latitud","Longitud"]):
    """ need a df with the coordenates in two columns
    imput dataframe, and the names of Longitud and Latitud
    output """
    geometria=[]
    for e in range(0,len(puntos)):
        datum=puntos[puntos.index==e][[names[0], names[1]]]
        punto=shape.Point(datum[names[1]], datum[names[0]])
        geometria.append(punto)
    puntos["geometry"]=geometria
    puntos=gpd.GeoDataFrame(puntos, crs="EPSG:4326")
    return (puntos)

In [3]:
def see_tables(conexion):
    cursor=conexion.cursor()
    query="""SELECT name FROM sqlite_master WHERE type='table';"""
    cursor.execute(query)
    return [ e[0] for e in cursor.fetchall()]

In [4]:
def ver_tabla (name):
    """ funcion para poder llamar a una de las tablas en formato df. necesita el nombre de la tabla entre comillas"""
    sql_query=f"SELECT * FROM {name}"
    df1 = pd.read_sql_query(sql_query, conexion)
    return df1

# Cleaning of the La Plata Museo DataBase

In [3]:
### paths needed of the DataBase of the Museum
path_data_base_museo="D:\00013-Tesis_Doctoral\2- Base de datos\Coreidae_A.xlsx"

In [4]:
### Open tables
t1=pd.read_excel(path_data_base_museo, sheet_name="Hoja1") ## they have the same headings
t2=pd.read_excel(path_data_base_museo, sheet_name="Hoja2") ## This have the same igut

In [11]:
### corrected table 1
# concat the colums to make the shape of localities studied " first: second, add"
t1["Locality"] = t1["State"]+": "+t1["Locality_wname"] # join two columns with the locality name
t1["Species"] = t1["Genus"]+" "+t1["Species"] # join the columns of genus and Species to have the column with the species name

In [35]:
### merge two tables with the necessary information
total= pd.concat([t1,t2], axis=0)

In [37]:
## delet the data that do not have identified species
total=total[total["Family"].notna()]
total=total[total["Species"].notna()]
total=total.fillna("")

In [72]:
### eliminamos cells duplicadas with the same information in each one
total = total.drop_duplicates()

In [76]:
# Save the information in a excel
total[total["Family"]=="Coreidae"].to_excel("Coreidae_Museo_la_plata.xlsx")

# Add Country in a dataframe with points

In [4]:
# Open the layer with the country of the world and the points records
path="D:/000033_GIS/Paises_Mundo.shp"
puntos="D:/00013-Tesis_Doctoral/2- Base de datos/Panstrongylus geniculatus.xlsx"
puntos_n=puntos
names_l=["Latitud","Longitud"]
puntos=pd.read_excel(puntos)
paises_shp=gpd.read_file(path)

In [6]:
# Create points as GeodataFrames and have the countries in the shapefile
puntos=CreateGDF_Points(puntos, names_l)
puntos=puntos.sjoin(paises_shp, how="inner", predicate='intersects')

In [9]:
# rewrite the archive with the points
puntos.to_excel(puntos_n)

# Uniformize with a specific name each archive used and drop duplicates

In [14]:
# Open folder with the databases of each species
folder="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae_AD"
#create a diccionary with the names to change of each column
diccionary_change={'genus': 'Genus', 
                   'Especie': 'Species',
                   "Pais":"Country",
                   'PAÍS':'Country', 
                   "Note":"Notes", 
                   "Genero":"Genus",  
                   "Notas":"Notes",
                   "Note":"Notes",
                   "Latitud":"Latitude", 
                   "Longitud":"Longitude",
                   "Localidad":"Locality",
                   "Pais":"Country",
                   "Año": "Year",
                  "Autor":"Reference",
                  "Author":"Reference",
                  "Unnamed: 8":"Notes"}

In [15]:
# using os explore all the archives of the folder
files = os.walk(folder)
for root, dirs, files in files:
    for name in files:
        file_excel=os.path.join(root, name) # obtein the nales of the path of each file seen in the folder
        #print(file_excel) # print to see with folder I am working on
        table=pd.read_excel(file_excel)
        table = table.rename(columns=diccionary_change)
        coordenadas=table
        coordenadas= coordenadas.drop_duplicates() # drop duplicates seened
        coordenadas.to_excel(file_excel, index=False) # replace the archive with the changes requires
        #coordenadas.drop(["Unnamed: 0"], axis=0)
        

# Join the Museum DataFrame with the species with the same name

In [ ]:
# Open the folder with the information of the museum and open the path with the folder with each archive separated
puntos="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae_Museo_la_plata.xlsx"
folder="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae_AD"
puntos=pd.read_excel(puntos)

In [ ]:
for root, dirs, files in os.walk (folder):
    for name in files:
        file_excel=os.path.join(root, name) # obtein the path of each archive in the folder
        table=pd.read_excel(file_excel) # open the table as a dataframe
        if name in puntos.Species.unique()+".xlsx": # Look if the name is listed in the species of the list of the museum
            name_sele=name[:-5]
            selected=puntos[puntos.Species==name_sele] # select the name
            puntos=puntos[puntos.Species!=name_sele]
            table=pd.concat([table, selected], axis=0) # concat both tables
        table.drop_duplicates() # drop duplicates
        table.to_excel(file_excel, index=False) # replace the archive with the information

In [ ]:
# save the escel of the La Plata Museum no studied in the records and not in bibiliography revised
puntos.to_excel("D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae_Museo_la_plata_no_archive.xlsx")

# Make a column with the political division of each archive to be revised and analysed

In [17]:
# open the layer with the politicas division in each lavel and read as a geodataframe
Level="D:/000033_GIS/political division/Level2.shp"
countries_gdf = gpd.read_file(Level) # the countries at level 2
# select the columns that will be added to the archive
countries_gdf=countries_gdf[["COUNTRY", "NAME_1", "NAME_2", "geometry"]] #### select names of each one
# create the way that will be shown the names of the locality
countries_gdf["Political Division"]=countries_gdf["COUNTRY"]+": "+countries_gdf["NAME_1"]+", "+countries_gdf["NAME_2"]

In [34]:
# folder with the databases of each especie
folder="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae_AD/Fal"
# names of the columns with the coordenades
coor_names=["Latitude", "Longitude"]

In [ ]:
datum=puntos[puntos.index==e][[coor_names[0], coornames[1]]]

In [ ]:
for root, dirs, files in os.walk (folder):
    for name in files:
        file_excel=os.path.join(root, name)
        print(file_excel)
        puntos=pd.read_excel(file_excel) # open the table
        #add the geometry to each species
        geometria=[]
        for e in range(0,len(puntos)):
            try:
                datum=puntos[puntos.index==e][[coor_names[0], coor_names[1]]]
                punto=shape.Point(datum[coor_names[1]], datum[coor_names[0]])
                geometria.append(punto)
            except Exception as e:
                geometria.append("Revise not found")
        puntos["geometry"]=geometria
        puntos=gpd.GeoDataFrame(puntos)
        # with sjoin drop all the entries with coordinates not recognized
        puntos=puntos.sjoin(countries_gdf , how="inner", predicate='intersects')
        puntos.drop(["COUNTRY", "NAME_1", "NAME_2", "geometry"], axis=1)
        puntos.to_excel(file_excel, index=False)

# Put the final_species_name

In [4]:
folder="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae"

In [ ]:
for root, dirs, files in os.walk (folder):
    for name in files:
        file_excel=os.path.join(root, name)
        index=name.split(".xlsx")[0]
        puntos=pd.read_excel(file_excel) # open the table
        puntos["Species_F"]=index
        puntos.to_excel(file_excel, index=False)

# Create a DB Archive with the information obtained

In [5]:
# path the archive of Database in Sqlite to the archive
conexion=sqlite3.connect("bd_chinches.db")
# open the folder with the archives with the ocurrences of each species
file="D:/00013-Tesis_Doctoral/2- Base de datos/Species/Coreidae"

In [6]:
# join all the tables of the species studied
Coreidea_final=pd.DataFrame() # create a df clean
files=os.listdir(file) # list the folder where we save the information needed
for e in files:
    path_gen=f"{file}/{e}"
    data=pd.read_excel(path_gen)
    Coreidea_final= pd.concat([Coreidea_final, data])
name="Records_per_species"
Coreidea_final.to_sql(name,conexion,if_exists='replace',index=False)
conexion.commit()

In [7]:
Coreidea_final=Coreidea_final.reset_index(drop=True)
#save the information as csv separated by tabulation
Coreidea_final.to_csv("Records_per_species_coreidae.csv",sep="\t", index=False)

# Create the general DB_to used

In [8]:
General_DB=pd.DataFrame({"Species_name":Coreidea_final.Species_F.unique(), "Family":"Coreidae"})

In [9]:
General_DB.to_excel("General_DB_Coreidae.xlsx", index=False)

In [10]:
General_DB

,Species_name,Family
0,Acanonicus hahni,Coreidae
1,Acanonicus riojana,Coreidae
2,Acanthocephala femorata,Coreidae
3,Acanthocephala surata,Coreidae
4,Acidomeria cincticornis,Coreidae
...,...,...
122,Zicca annulata,Coreidae
123,Zicca cornuta,Coreidae
124,Zicca nigropunctata,Coreidae
125,Zicca stali,Coreidae
